In [13]:
import requests
from bs4 import BeautifulSoup
import time
import re
# import pandas as pd
import pandas as pd
# from emoji import replace_emoji #for remove emoji from description
from numpy import nan 
from numpy.random import default_rng
import os

In [14]:
class Extract_link_page:
    def __init__(self,n_page , primary_link) :
        self.number_of_pages = n_page
        self.primary_link = primary_link
    def Extract(self,page_count_auto_save=50,save_end=False,show_time=False ,return_links=False, show_page=False,show_duplicate=False,random_request=False):
        
        if show_time: t=time.time() ; t1=time.time()
        
        link_page=[]
        file_name_list=[]
           
        for i in range(0,self.number_of_pages):
                # time.sleep(1)
                # if break_>10:break
                html=requests.get(self.primary_link+str(i))
                if html.status_code == 200:

                    soup=BeautifulSoup(html.content,'html.parser')
                    
                    if show_page :print("page :" , i)

                    links=soup.find_all('a',class_="kilid-listing-card flex-col al-start ng-star-inserted", href=True)
                    if len(links)<10: print("tedad link break") ; continue

                    # else:print(links)
                    for a in links:
                        # sleep(0.2)
                        if a["href"] in link_page:
                            if show_duplicate:print("duplicate in page :",i)
                            n_duplicate +=1
                            continue
                        else:
                            link_page.append(a["href"])
                        # print("Found the URL:", a['href'])
                if (i % page_count_auto_save == 0) and (i!=0) and(page_count_auto_save != 0):
                 # print("link_total_%s.txt"%time.strftime("%H:%M:%S"))
                    file_name = "link_total_%s.txt"%time.strftime("%H:%M:%S").replace(":","_")

                    with open(file_name,"w") as f:
                        f.write(",".join(link_page))
                        f.close()          
                    file_name_list.append(file_name)
                
                if len(file_name_list) >1:
                       os.remove(file_name_list[-2])
                
                  
                if random_request:    
                    rand=  default_rng().uniform(6,11) 
                    time.sleep(rand)
                    print('random request time = %d' %rand) 
                if show_time:
                    print(f'time this request = {round(time.time()-t,3)}')
                    t=time.time()
                    print(f"total time = {round(time.time()-t1,3)}")
        if save_end:
            with open("link_total_%s.txt"%time.strftime("%H:%M:%S").replace(":","_"),"w") as f:
                    f.write(",".join(link_page))
                    f.close()  
            
        if return_links:
            return link_page
            # print(link_home)
        
    

In [15]:
# t1 = Extract_link_page(2, "https://kilid.com/buy/tehran?listingTypeId=1&page=")
# t1.Extract(n_auto_saving=50 ,show_duplicate=True,show_page=True,show_time=True,random_request=True,return_links=True,)

In [16]:
class Extract_informations:
      merge_dic=lambda x,y:x|y

      def replace_chars(self,text):
         for ch in ['/','`','*','_','{','}','[',']','(',')','>','#','+','-','!','$','\'',"//"]:
              if ch in text:
                  text = text.replace(ch," ")
         return text

      def try_css_selector(self,soup,css):

          try:
              return self.replace_chars(soup.select_one(css).text.strip())

          except AttributeError:
              return None
      
      def extract_location(self,soup,css):
         
         try:
            location=dict()
            total_loc=[]
            total_loc = self.try_css_selector(soup,css).split("،")
         except: 
            location["region"]=None
            location["city"]=None
            location["address"]=None
            return location
         else:
            region=[]
            for i in total_loc[1].strip():
               if i.isdigit():
                  region.append(i)
            try: location["city"]=total_loc[0].strip()
            except :  location["city"]=None
            try:  location["region"]="".join(region)
            except: location["region"]=None
            try:  location["address"]=total_loc[2].strip()
            except:  location["address"]=None

            return location
         

      def extract_features(self,soup,css):
         features=dict()
         features2=dict()
         name=[]
         nums=[]
         try:
            for i in self.try_css_selector(soup,css).split():
               if i.isalpha():
                  name.append(i)
               else:nums.append(i)

            features={k:v for k,v in zip(name,nums)}

            features2["parking"] = features.get("پارکینگ",None)    
            features2["Meterage"] = features.get("متر",None)    
            features2["bedroom "] = features.get("خوابه",None)    
            features2["age_year"] = features.get("ساله",None)    

         except:
            features2["parking"] = None 
            features2["meterage"] = None
            features2["bedroom "] = None    
            features2["age_year"] = None    

         return features2


      def extract_persian_text(self,soup, css):
         try:
            string_=str(soup.select_one(css))

            facilities=[]
            pattern =r'.*>([\u0600-\u06FF]+\s*[\u0600-\u06FF]*)<.*'
            while True:
                try:
                    y=re.search(pattern,string_)
                    if y.groups():
                        facilities.append(y.groups()[0])
                        string_=string_.replace(facilities[-1],"")          
                    else:break

                except  : break


            return " , ".join(facilities)

         except: return None

      def extract_number(self,soup,css):
         try:
            for i in self.try_css_selector(soup=soup,css=css).split():
               if i.isdigit():
                  return int(i)
         except:
            return None


      def details(self,href):
         html = requests.get(href)

         if html.status_code == 200:
            soup = BeautifulSoup(html.content , "html.parser")

            home=dict() 
            home["title"]= self.try_css_selector(soup,".single-data__info")  
            try:
               home["total_price"]=self.replace_chars(self.try_css_selector(soup,".single-data__container.ng-star-inserted").split()[2])
            except:
               home["total_price"]=None

            home["price_per_meter"]=self.extract_number(soup=soup,css=".ng-star-inserted+ .single-data__container")

            # home=self.merge_dic(home,self.extract_location(soup=soup , css=".single-data__location span"))
            loc = self.extract_location(soup,".single-data__location span")
            home=home|loc
            # home=self.merge_dic(home,self.extract_features(soup,".single-data__location span"))
            features= self.extract_features(soup,".single-data__container--attributes")
            home=home|features
            home["facilities"] = self.extract_persian_text(soup,".ng-trigger-slideDown")
            home["adviser"] = self.try_css_selector(soup=soup , css= ".single-sticky__department__user-name")
            home["real_estate"] =self.try_css_selector(soup=soup,css=".single-sticky__department__name")
            home["ad_code"] = self.extract_number(soup=soup,css=".single-sticky__info__item:nth-child(1)")
            # home["description"] = self.extract_persian_text(soup,".single-description")
            # try: home["description"] = replace_emoji(soup.select_one(".single-description").text.strip())
            # except:   home["description"] = None
               
            
            return home

                  
      def scrap_with_start_end(self,primary_link , list_links ,start,end,random_request=False,show_time=False):
               
               files_name=[]
               df=pd.DataFrame()
               if show_time: t=time.time() ; t1=time.time()   
               for url in range(start,end):
                  
                  print("link number = %d" %url)
                  href = primary_link+list_links[url]
                  

                  output=pd.DataFrame(self.details(href),index=[0])
                   
                  df=pd.concat([df,output],axis=0,ignore_index=True)
                   
                  if random_request:    
                     rand=  default_rng().uniform(random_request[0],random_request[1]) 
                     print('random request time = %d' %rand)    
                     time.sleep(rand)
                  if show_time:
                     print(f'time this request = {round(time.time()-t,3)}')
                     t=time.time()
                     print(f"total time = {round(time.time()-t1,3)}")
                     
                  myfile = "homes_not_clean_%s.xlsx"%time.strftime("%H:%M:%S").replace(":","_")
                  df.to_excel(myfile,index=None)

                  # print(myfile +" "+len(files_name) )
                  files_name.append(myfile)

                  # print(len(files_name))
                  if len(files_name) >1: 
                     try:
                        os.remove(files_name[-2])
                        
                     except: print("file not delete")
                  
                  
                  
               return df
            
      def data_cleaning(self,df):
        
         x=[]
         
         for i in df.facilities[df.facilities.notna()].str.split(" , "):
           for j in i:
         
             x.append(j)
         x=list(set(x))
         len(x)
         y ="""'sauna',
           'proportionate shares',
           'roof garden',
           'balcony',
           'sports hall',
           'exchange',
           'guardian',
           'agreed price',
           'Elevator',
           'Jacuzzi',
           'have loan',
           'conference hall',
           'newly built',
           'mall',
           'lobby',
           'remote door',
           'air conditioning',
           'pool',
           'Central antenna',
           'Warehouse'
           """

         dict_unique_features=dict(zip(x,[i.strip().replace(" ","_") for i in y.replace("\n","").replace("'","").split(", ")]))
         dict_unique_features

         temp=dict()
         for i in range(df.shape[0]):
           list_features=[]
           for j in dict_unique_features.keys():
             if df.facilities[i] is nan: 
                 list_features.append(nan)
                 continue
              
             if j in df.facilities[i].split(" , "):
               list_features.append(1)
             else :
               list_features.append(0)
           temp[i]=list_features
         facilities=pd.DataFrame(temp.values(),columns=dict_unique_features.values(),index=temp.keys())

         
         del temp
         del list_features
         
         df2=pd.concat([df,facilities],axis=1).drop("facilities",axis=1)
         myfile = "homes_cleaned_%s.xlsx"%time.strftime("%H:%M:%S").replace(":","_")
         df2.to_excel(myfile,index=None)
         
         return df2


         


In [17]:
a=list((2,3,4))
a[-2]

3

In [19]:
with open("link_total_19_15_32.txt","r") as f:
  
      link_homes=f.read().split()
      
homes=Extract_informations()

random_request_time=(7,23)
# change start , end 👇👇👇👇
start  = 10000 
end = start +1000   # len(link_homes)
df=homes.scrap_with_start_end(primary_link="https://kilid.com" , list_links= link_homes ,start= start, end=end ,random_request=random_request_time,show_time=True)
df_clean = homes.data_cleaning(df)

# df_clean.to_excel("homes.xlsx",index=None)


link number = 10000
random request time = 16
time this request = 17.864
total time = 17.864
link number = 10001
random request time = 19
time this request = 21.91
total time = 39.774
lennnn##
File Deleted successfully
link number = 10002
random request time = 9
time this request = 10.703
total time = 50.476
lennnn##
File Deleted successfully
link number = 10003
random request time = 15
time this request = 16.812
total time = 67.288
lennnn##
File Deleted successfully
link number = 10004
random request time = 12
time this request = 17.356
total time = 84.644
lennnn##
File Deleted successfully
link number = 10005
random request time = 18
time this request = 19.629
total time = 104.275
lennnn##
File Deleted successfully
link number = 10006
random request time = 17
time this request = 19.192
total time = 123.468
lennnn##
File Deleted successfully
link number = 10007
random request time = 10
time this request = 14.813
total time = 138.28
lennnn##
File Deleted successfully
link number = 10008